We will use Supabase as the PGVector example database</br>
Sign up with Supabase</br>
Enable the "vector" extension</br>
Go to Langchain site for the info of Supabase vector store: </br>
https://python.langchain.com/docs/integrations/vectorstores/supabase

Run the table creation script in the SQL Editor
```
-- Enable the pgvector extension to work with embedding vectors
create extension if not exists vector;

-- Create a table to store your documents
create table
  documents (
    id uuid primary key,
    content text, -- corresponds to Document.pageContent
    metadata jsonb, -- corresponds to Document.metadata
    embedding vector (1536) -- 1536 works for OpenAI embeddings, change if needed
  );

-- Create a function to search for documents
create function match_documents (
  query_embedding vector (1536),
  filter jsonb default '{}'
) returns table (
  id uuid,
  content text,
  metadata jsonb,
  similarity float
) language plpgsql as $$
#variable_conflict use_column
begin
  return query
  select
    id,
    content,
    metadata,
    1 - (documents.embedding <=> query_embedding) as similarity
  from documents
  where metadata @> filter
  order by documents.embedding <=> query_embedding;
end;
$$;
```

In [ ]:
%pip install supabase
import os
import getpass

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key: ")

In [ ]:
os.environ["SUPABASE_URL"] = getpass.getpass("Enter Supabase URL: ")

In [ ]:
os.environ["SUPABASE_SERVICE_KEY"] = getpass.getpass("Enter Supabase Service Key: ")

In [ ]:
from supabase.client import Client, create_client
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import SupabaseVectorStore

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

embeddings = OpenAIEmbeddings()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

loader = TextLoader("./state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
vector_store = SupabaseVectorStore.from_documents(docs, embeddings, client=supabase, table_name="documents", query_name="match_documents")
# vector_store = SupabaseVectorStore(embedding=embeddings, client=supabase, table_name="documents", query_name="match_documents")

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
matched_docs = vector_store.similarity_search(query)

In [ ]:
print(matched_docs[0].page_content)

In [ ]:
matched_docs = vector_store.similarity_search_with_relevance_scores(query)

In [ ]:
matched_docs[0]